 # Numba

 Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code. It translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

 For this tutorial, we will be following the documentation [here](https://numba.readthedocs.io/en/stable/index.html).

 To install it, as always we just need to type:

In [1]:
! pip install numba

Numba is a just-in-time compiler for Python that works best on code that uses NumPy arrays and functions, and loops. The most common way to use Numba is through its collection of decorators that can be applied to your functions to instruct Numba to compile them. When a call is made to a Numba-decorated function it is compiled to machine code “just-in-time” for execution and all or part of your code can subsequently run at native machine code speed!

## Will Numba work for my code?

This depends on what your code looks like, if your code is numerically orientated (does a lot of math), uses NumPy a lot and/or has a lot of loops, then Numba is often a good choice. In these examples we’ll apply the most fundamental of Numba’s JIT decorators, @jit, to try and speed up some functions to demonstrate what works well and what does not.

Numba works well on code that looks like this:

In [2]:
from numba import jit

x = np.arange(100).reshape(10, 10)

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

print(go_fast(x))

NameError: name 'np' is not defined

## What is nopython mode?

The Numba @jit decorator fundamentally operates in two compilation modes, nopython mode and object mode. In the go_fast example above, nopython=True is set in the @jit decorator; this is instructing Numba to operate in nopython mode. The behaviour of the nopython compilation mode is to essentially compile the decorated function so that it will run entirely without the involvement of the Python interpreter. This is the recommended and best-practice way to use the Numba jit decorator as it leads to the best performance.

## Numpy suported features

Numba provides a huge speed gain, but not all NumPy functions work under Numba. Make sure you check the list before executing your code. You can find it [here](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html).

In [58]:
import time
A = np.arange(1,100000000,1)
def range_test(A):
    s = 0
    for i in range(A.shape[0]):
        s += A[i]
    return s

tic = time.time()
range_test(A)
toc = time.time()
print("Elapsed time is :", toc-tic)


C:\Users\Sergi\AppData\Local\Temp\ipykernel_7160\2023199476.py:8: RuntimeWarning: overflow encountered in scalar add
  s += A[i]


Elapsed time is : 17.613186836242676


In [3]:
from numba import njit
import time
import numpy as np
A = np.arange(1,100000000,1)

@njit()
def numba_test(A):
    s = 0
    for i in range(A.shape[0]):
        s += A[i]**(-0.4)
    return s

tic = time.time()
numba_test(A)
toc = time.time()
print("Elapsed time is :", toc-tic)

Elapsed time is : 5.355860233306885


In [4]:
from numba import prange

@njit(parallel=True)
def prange_test(A):
    s = 0
    for i in prange(A.shape[0]):
        s += A[i]**(-0.4)
    return s

tic = time.time()
prange_test(A)
toc = time.time()
print("Elapsed time is :", toc-tic)

Elapsed time is : 1.2401206493377686


In [5]:
tic = time.time()
np.sum(A**(-0.4))
toc = time.time()
print("Elapsed time is :", toc-tic)

Elapsed time is : 4.032361030578613


In [10]:
@njit(parallel=True)
def func1(A):
    np.sum(A**(-0.4))

tic = time.time()
func1(A)
toc = time.time()
print("Elapsed time is :", toc-tic)

Elapsed time is : 1.0232255458831787
